##  Chameleon Facility Port

This notebook demonstrates how to stitch experiment spanning [Chameleon](https://www.chameleoncloud.org/) and FABRIC.

### Set Chameleon Environment

FABRIC should already be configured in your Jupyter environment. If this is your first time using FABRIC, may need to follow the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.

Set the following environment vars to the values found in your custom Chameleon-openrc.sh file. You can obtain your Chameleon-openrc.sh from Chameleon using these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script).  The expected password is not your regular Chameleon password. Instead, you need to create a Chameleon CLI password by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication)

In [1]:
import os

#Chameleon
#os.environ["OS_PROJECT_ID"]="4140e5f9f65545dbb9f0bdc90ef68d23"
#FABRIC@Chameleon
os.environ["OS_PROJECT_ID"]="ae76673270164b048b59d3bd30676721"

os.environ["OS_USERNAME"]="pruth"
os.environ["OS_PASSWORD"]='Ren72jUGvWMq'
os.environ["OS_AUTH_URL"]='https://chi.uc.chameleoncloud.org:5000/v3'
os.environ["OS_IDENTITY_API_VERSION"]='3'
os.environ["OS_INTERFACE"]='public'
os.environ["OS_PROTOCOL"]="openid"
os.environ["OS_AUTH_TYPE"]="v3oidcpassword"
os.environ["OS_IDENTITY_PROVIDER"]="chameleon"
os.environ["OS_DISCOVERY_ENDPOINT"]="https://auth.chameleoncloud.org/auth/realms/chameleon/.well-known/openid-configuration"#
os.environ["OS_CLIENT_ID"]="keystone-uc-prod"
os.environ["OS_ACCESS_TOKEN_TYPE"]="access_token"
os.environ["OS_CLIENT_SECRET"]="none"
os.environ["OS_REGION_NAME"]="CHI@UC"


#os.environ["OS_AUTH_TYPE"]='v3applicationcredential'
#os.environ["OS_AUTH_URL"]='https://chi.uc.chameleoncloud.org:5000/v3'
#os.environ["OS_IDENTITY_API_VERSION"]='3'
#os.environ["OS_REGION_NAME"]="CHI@UC"
#os.environ["OS_INTERFACE"]='public'
#os.environ["OS_APPLICATION_CREDENTIAL_ID"]='9f0c6c736cf54577a0f6666b357da1ee'
#os.environ["OS_APPLICATION_CREDENTIAL_SECRET"]='fHd_d63IfpGKo6gsFTCSnZ2gLXmzE1dQX1jECVU71VwZQhmsXi348ih520hQMFT19AcVYjAZ5n9QJcu6HdHGUA'


### Import Libraries

In [2]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease 
from chi.server import *
from chi.lease import *
from chi.network import *

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

### Chameleon Experiment Variable

In [3]:
# Chameleon Config
chameleon_prefix =  "fabric_stitch12"
chameleon_server_name = chameleon_prefix+'Server'
chameleon_network_name = chameleon_prefix+'Net'
chameleon_subnet_name = chameleon_prefix+'Subnet'
chameleon_router_name = chameleon_prefix+'Router'
chameleon_lease_name = chameleon_prefix+'Lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_node_type="compute_cascadelake_r"
chameleon_physical_network='physnet1'
chameleon_stitch_provider='fabric'
chameleon_server_count=1
chameleon_key_name='my_chameleon_key'

### FABRIC Experiment Variables

In [4]:
# Create a FABlib manager
fablib = fablib_manager()
fablib.show_config()

# FABRIC Config
fabric_slice_name='chameleon_stitch'
fabric_node_name='node1'

fabric_node_image='default_rocky_8'
fabric_site=fablib.get_random_site()
print(f'fabric_site: {fabric_site}')

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


fabric_site: MICH


### Network Variables

The stitched network spans both Chameleon and FABRIC. The network needs a common subnet but distinct IP address allocation pools.

In [5]:
#Network Config
subnet = IPv4Network("192.168.100.0/24")

fabric_allocation_pool_start=IPv4Address('192.168.100.200')
fabric_allocation_pool_end=IPv4Address('192.168.100.250')
fabric_available_ips=[]
for ip_int in range(int(fabric_allocation_pool_start),int(fabric_allocation_pool_end)+1):
    fabric_available_ips.append(IPv4Address(ip_int))
    
chameleon_allocation_pool_start='192.168.100.100'
chameleon_allocation_pool_end='192.168.100.150'
chameleon_gateway_ip='192.168.100.1'

## Create the Chameleon Network

#### Create a Lease at Chicago

In [6]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#chi.lease.add_node_reservation(reservation_list, count=chameleon_server_count, node_type=chameleon_node_type)
#chi.lease.add_fip_reservation(reservation_list, count=2)

reservation_list.append(
        {
            "resource_type": "network",
            "network_name": chameleon_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", chameleon_stitch_provider]
            ),
        }
)

# Create the lease
chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                  reservations=reservation_list,
                                  start_date=start_date,
                                  end_date=end_date)
    
#Print the lease info
#print(json.dumps(chameleon_lease, indent=2))
#chameleon_compute_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
#chameleon_floatingip_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

#print(f"chameleon_compute_reservation_id: {chameleon_compute_reservation_id}")
print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")
#print(f"chameleon_floatingip_reservation_id: {chameleon_floatingip_reservation_id}")

chameleon_network_reservation_id: 19651bc1-6bb9-4c47-970f-e93585089c96


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [13]:
network_vlan = None
while network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

Chameleon Network is not ready. Trying again!
Chameleon Network is not ready. Trying again!
Chameleon Network is not ready. Trying again!
Chameleon Network is not ready. Trying again!
Chameleon Network is not ready. Trying again!
Chameleon Network ID: 5bdc9011-941f-4372-8490-a7dd43d40464
network_vlan: 3307


#### Add a subnet to the reserved network

In [14]:
chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(subnet),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip)

print(json.dumps(chameleon_subnet, indent=2))

{
  "id": "51b4ef4d-74fb-41b5-9ac0-545729f660cf",
  "name": "fabric_stitch12Subnet",
  "tenant_id": "ae76673270164b048b59d3bd30676721",
  "network_id": "5bdc9011-941f-4372-8490-a7dd43d40464",
  "ip_version": 4,
  "subnetpool_id": null,
  "enable_dhcp": true,
  "ipv6_ra_mode": null,
  "ipv6_address_mode": null,
  "gateway_ip": "192.168.100.1",
  "cidr": "192.168.100.0/24",
  "allocation_pools": [
    {
      "start": "192.168.100.100",
      "end": "192.168.100.150"
    }
  ],
  "host_routes": [],
  "dns_nameservers": [],
  "description": "",
  "service_types": [],
  "tags": [],
  "created_at": "2022-11-10T19:34:10Z",
  "updated_at": "2022-11-10T19:34:10Z",
  "revision_number": 0,
  "project_id": "ae76673270164b048b59d3bd30676721"
}


#### Add a Router

In [15]:
chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')

print(json.dumps(chameleon_router, indent=2))

{
  "id": "78a62507-8258-47fa-b7c8-1a6ac44613a3",
  "name": "fabric_stitch12Router",
  "tenant_id": "ae76673270164b048b59d3bd30676721",
  "admin_state_up": true,
  "status": "ACTIVE",
  "external_gateway_info": {
    "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
    "external_fixed_ips": [
      {
        "subnet_id": "c3950603-9e04-4cc5-be8d-1efbfe59fc0a",
        "ip_address": "192.5.86.191"
      }
    ],
    "enable_snat": true
  },
  "description": "",
  "availability_zones": [],
  "availability_zone_hints": [],
  "routes": [],
  "flavor_id": null,
  "tags": [],
  "created_at": "2022-11-10T19:34:10Z",
  "updated_at": "2022-11-10T19:34:11Z",
  "revision_number": 3,
  "project_id": "ae76673270164b048b59d3bd30676721"
}


#### Attach the Router and Subnet

In [16]:
chi.network.add_subnet_to_router_by_name(chameleon_router_name, chameleon_subnet_name)

{'id': '78a62507-8258-47fa-b7c8-1a6ac44613a3',
 'tenant_id': 'ae76673270164b048b59d3bd30676721',
 'port_id': '92ed2e5b-9be6-4d34-9963-f13ba5effced',
 'network_id': '5bdc9011-941f-4372-8490-a7dd43d40464',
 'subnet_id': '51b4ef4d-74fb-41b5-9ac0-545729f660cf',
 'subnet_ids': ['51b4ef4d-74fb-41b5-9ac0-545729f660cf']}

## Create FABRIC Node and Link

In [17]:
try:
    fabric_site='MICH'
    fabric_slice_name="stitch12"
    
    #Create a slice
    fabric_slice = fablib.new_slice(name=fabric_slice_name)
    
    fabric_node = fabric_slice.add_node(name=fabric_node_name, site=fabric_site, image=fabric_node_image)
    #fabric_node_iface = fabric_node.add_component(model='NIC_ConnectX_5', name=f"nic1").get_interfaces()[0]
    fabric_node_iface = fabric_node.add_component(model='NIC_Basic', name=f"nic1").get_interfaces()[0]
    
    fabric_facility_port = fabric_slice.add_facility_port(name='Chameleon-StarLight', site='STAR', vlan=str(network_vlan))
    fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]

    fabric_net = fabric_slice.add_l2network(name=f'net_facility_port', interfaces=[fabric_node_iface,fabric_facility_port_iface], type='L2STS')

    #Submit the Request
    fabric_slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


Retry: 8, Time: 191 sec


ID,98f6f141-a9f1-4a06-a5bb-ac2626601c0b
Name,stitch12
Lease Expiration (UTC),2022-11-11 19:34:13 +0000
Lease Start (UTC),2022-11-10 19:34:14 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8b05e641-88df-46e9-8ae8-f83608d1bdb1,node1,2,8,10,default_rocky_8,qcow2,mich-w2.fabric-testbed.net,MICH,rocky,2607:f018:110:11:f816:3eff:fe87:e726,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2607:f018:110:11:f816:3eff:fe87:e726,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
270f036a-4c50-4485-83d4-22383f915eba,net_facility_port,L2,L2STS,None,None,None,Active,



Time to stable 191 seconds
Running post_boot_config ... Time to post boot config 194 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
node1-nic1-p1,node1,net_facility_port,100,None,02:3C:4D:C1:F3:A1,eth1,eth1



Time to print interfaces 196 seconds


## Configure the FABRIC VM's IP Address

In [18]:
try:        
    fabric_node = fabric_slice.get_node(name=fabric_node_name)   
    
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port') 
    fabric_node_addr = fabric_available_ips.pop(0)
    print(f"fabric_node_addr: {fabric_node_addr}")
    fabric_node_iface.ip_addr_add(addr=fabric_node_addr, subnet=subnet)
    
    stdout, stderr = fabric_node.execute(f'ip addr show {fabric_node_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")

fabric_node_addr: 192.168.100.200
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:3c:4d:c1:f3:a1 brd ff:ff:ff:ff:ff:ff
    inet 192.168.100.200/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::3f2e:d72b:16c5:f871/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:3c:4d:c1:f3:a1 brd ff:ff:ff:ff:ff:ff
    inet 192.168.100.200/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::3f2e:d72b:16c5:f871/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever



## (Optional) Test the Link

We can test the facility port link by pinging the Chameleon gateway IP from the FABRIC VM.

In [19]:
try:
    fabric_node = fabric_slice.get_node(name=fabric_node_name)     
    fabric_node_iface = fabric_node.get_interface(network_name=f'net_facility_port') 


    stdout, stderr = fabric_node.execute(f'ping -c 5 {chameleon_gateway_ip}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

PING 192.168.100.1 (192.168.100.1) 56(84) bytes of data.
64 bytes from 192.168.100.1: icmp_seq=1 ttl=64 time=15.7 ms
64 bytes from 192.168.100.1: icmp_seq=2 ttl=64 time=7.55 ms
64 bytes from 192.168.100.1: icmp_seq=3 ttl=64 time=7.56 ms
64 bytes from 192.168.100.1: icmp_seq=4 ttl=64 time=7.55 ms
64 bytes from 192.168.100.1: icmp_seq=5 ttl=64 time=7.56 ms

--- 192.168.100.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 7.551/9.181/15.686/3.253 ms
PING 192.168.100.1 (192.168.100.1) 56(84) bytes of data.
64 bytes from 192.168.100.1: icmp_seq=1 ttl=64 time=15.7 ms
64 bytes from 192.168.100.1: icmp_seq=2 ttl=64 time=7.55 ms
64 bytes from 192.168.100.1: icmp_seq=3 ttl=64 time=7.56 ms
64 bytes from 192.168.100.1: icmp_seq=4 ttl=64 time=7.55 ms
64 bytes from 192.168.100.1: icmp_seq=5 ttl=64 time=7.56 ms

--- 192.168.100.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4007ms
rtt min/avg/max/mdev = 7.551/9.

## Start Chameleon Servers

In [ ]:
#import chi.server
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    # Create the server
    server = chi.server.create_server(server_name, 
                                  reservation_id=chameleon_compute_reservation_id, 
                                  network_name=chameleon_network_name, 
                                  image_name=chameleon_image_name,
                                  key_name=chameleon_key_name
                                 )
# Wait until the server is active
#chi.server.wait_for_active(server.id)

## Get the Chameleon Server Fixed IPs

In [ ]:
#get fixed ips
fixed_ips={}
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Test the Link by Pinging All Chameleon Nodes

In [ ]:
for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')
    
    stdout, stderr = fabric_node.execute(f'ping -c 5 {fixed_ip}')
    print (stdout)
    print (stderr)

# Clean Up 

## Delete Chameleon Resources

Delete the servers

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i}"
    chi.server.delete_server(get_server_id(server_name))

#### De-configure Network

In [ ]:
router_id = chameleon_router['id']
subnet_id = chameleon_subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

## Release Lease

In [ ]:
chi.lease.delete_lease(chameleon_lease['id'])

## Delete FABRIC Slice

In [ ]:
try:
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")